In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Qdrant
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [3]:
loader = PyPDFLoader("../coba_qa_instruct/Kurikulum Teknik Informatika - FILKOM 2020 v1.9.pdf")
documents = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex = False,
)

docs = text_splitter.split_documents(documents)

In [12]:
embeddings = HuggingFaceEmbeddings(
    model_name = "firqaaa/indo-sentence-bert-base",
    cache_folder = "./cache-indo-sentence/"
)

In [14]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="./local_qdrant",
    collection_name="kurikulum-tif",
)

In [17]:
query = "Apa saja yang dipelajari pada mata kuliah pemrosesan bahasa alami?"
found_docs = qdrant.similarity_search(query)
found_docs

[Document(page_content='di dunia nyata yang menarik, khususnya masalah di dunia pendidikan. Selain itu, mata kuliah ini dirancang sebagai pengantar sains data dalam konteks pemrograman menggunakan bahasa pemrograman Python. CAPAIAN PEMBELAJARAN MATA KULIAH (CPMK) M1 Mampu mendeskripsikan dan menjelaskan tentang data, perjalanan data, dan bagaimana mengeksplorasi data M2 Mampu mendeskripsikan dan menjelaskan tentang sains data   M3 Mampu mendeskripsikan dan menjelaskan konsep utama, alat, algoritme, dan aplikasi sains data  M4 Mampu menjelaskan metodologi sains data', metadata={'source': '../coba_qa_instruct//Kurikulum Teknik Informatika - FILKOM 2020 v1.9.pdf', 'page': 112}),
 Document(page_content='Algoritma dan Struktur Data  NAMA MATA KULIAH Nama Pemrosesan Bahasa Alami Kode MK CIF62055 Kredit (sks) 3 Semester Genap DESKRIPSI MATA KULIAH Mata kuliah ini didesain untuk membantu mahasiswa dalam mempelajari dasar-dasar pemrosesan bahasa alami atau natural language processing (NLP) supa

In [21]:
from langchain.llms import OpenAI
import getpass
import os

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

llm = OpenAI(
    model_name="gpt-3.5-turbo",
)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=qdrant.as_retriever())

/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/data/home/abiyyu/miniconda3/envs/biyu/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [23]:
qa.run("Apa saja yang dipelajari pada mata kuliah pemrosesan bahasa alami?")

'Pada mata kuliah pemrosesan bahasa alami, dipelajari dasar-dasar pemrosesan bahasa alami, seperti natural language understanding, natural language generation, dan natural language inference. Juga dipelajari berbagai tugas NLP, termasuk POS tagging kata, ekstraksi informasi, penghilangan ambiguitas makna, parsing probabilistik, dan pencarian informasi. Mata kuliah ini juga membahas aplikasi NLP seperti analisis sentimen dan question answering system.'

In [26]:
qa.run("Siapa pengarah tim penyusun kurikulum? jawab dengan bahasa indonesia!")

'Pengarah tim penyusun kurikulum adalah Wayan Firdaus Mahmudy, S.Si., M.T., Ph.D.'

In [28]:
print(qa.run("Siapa saja tim penyusun kurikulum? jawab dengan bahasa indonesia!"))

Tim penyusun kurikulum terdiri dari:
- Pengarah Wayan Firdaus Mahmudy, S.Si., M.T., Ph.D.
- Penanggung Jawab Achmad Basuki, S.T., M.MG., PhD.
- Ketua Adhitya Bhawiyuga, S.Kom., M.Sc.
- Anggota Tri Astoto Kurniawan, S.T., M.T., Ph.D.
- Agus Wahyu Widodo, S.T., M.Cs.
- Achmad Arwan, S.Kom., M.Kom.
- Agi Putra Kharisma, S.T., M.T.
- Drs. Muh. Arif Rahman, M.Kom.
- Putra Pandu Adikara, S.Kom., M.Kom.
- Ratih Kartika Dewi, S.T., M.Kom.
- Sigit Adinugroho, S.Kom., M.Sc.
- Yuita Arum Sari, S.Kom., M.Kom.
- Komang Candra Brata, S.Kom., M.T., M.Sc.
- Adam Hendra Brata, S.Kom., M.T., M.Sc.
